# Data Science Boot Camp

## Introduction to Pandas Part 2

## Data Wrangling with Pandas

* Now that we have been exposed to the basic functionality of Pandas, lets explore some more advanced features that will be useful when addressing more complex data management tasks.<br>
<br>
* As most statisticians/data analysts will admit, often the lion's share of the time spent implementing an analysis is devoted to preparing the data itself, rather than to coding or running a particular model that uses the data.<br>
<br>
* This is where Pandas and  Python's standard library are beneficial, providing high-level, flexible, and efficient tools for manipulating your data as needed.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 25)

### Date/Time data handling

* Date and time data are inherently problematic:<br>
    * There are an unequal number of days in every month, an unequal number of days in a year (due to leap years)<br>
    <br>
    * Time zones that vary over space.<br>

* But information about time is essential in many analyses, particularly in the case of time series analysis.

* The `datetime` built-in library handles temporal information down to the nanosecond.

In [210]:
from datetime import datetime

In [211]:
now = datetime.now()
now

datetime.datetime(2018, 4, 9, 12, 11, 21, 285235)

In [212]:
now.day

9

In [213]:
now.weekday()

0

* In addition to `datetime` there are simpler objects for date and time information only, respectively.

In [214]:
from datetime import date, time

In [215]:
time(3, 24)

datetime.time(3, 24)

In [216]:
date(1987, 8, 8)

datetime.date(1987, 8, 8)

* Having a custom data type for dates and times is convenient because we can perform operations on them easily. For example, we may want to calculate the difference between two times:

In [217]:
my_age = now - datetime(1987, 8, 8)
my_age

datetime.timedelta(11202, 43881, 285235)

In [218]:
my_age.days/365.

30.69041095890411

## Merging and joining DataFrame objects

* Data contained in pandas objects can be combined together in a number of built-in ways:

* `pandas.merge` connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.

* `pandas.concat` glues or stacks together objects along an axis.

* `combine_first` instance method enables splicing together overlapping data to fill
in missing values in one object with values from another.

### Database-style DataFrame Merges

* Merge or join operations combine data sets by linking rows using one or more keys.

In [3]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [220]:
pd.merge(df1, df2)

   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

* Note that I didn’t specify which column to join on. If not specified, merge uses the overlapping column names as the keys.

* If the column names are different in each object, you can specify them separately:

In [221]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

   data1 lkey  data2 rkey
0      0    b      1    b
1      1    b      1    b
2      6    b      1    b
3      2    a      0    a
4      4    a      0    a
5      5    a      0    a

* You probably noticed that the 'c' and 'd' values and associated data are missing from the result.<br>
<br>
* By default merge does an 'inner' join; the keys in the result are the intersection.

* Other possible options are 'left', 'right', and 'outer'.

In [222]:
pd.merge(df1, df2, how='outer')

   data1 key  data2
0    0.0   b    1.0
1    1.0   b    1.0
2    6.0   b    1.0
3    2.0   a    0.0
4    4.0   a    0.0
5    5.0   a    0.0
6    3.0   c    NaN
7    NaN   d    2.0

In [223]:
pd.merge(df1, df2, on='key', how='left')

   data1 key  data2
0      0   b    1.0
1      1   b    1.0
2      2   a    0.0
3      3   c    NaN
4      4   a    0.0
5      5   a    0.0
6      6   b    1.0

In [224]:
pd.merge(df1, df2, on='key', how='right')

   data1 key  data2
0    0.0   b      1
1    1.0   b      1
2    6.0   b      1
3    2.0   a      0
4    4.0   a      0
5    5.0   a      0
6    NaN   d      2

* In some cases, the merge key or keys in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key.

In [225]:
df5 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
df5

   group_val
a        3.5
b        7.0

In [226]:
pd.merge(df4, df5, left_on='rkey', right_index=True)

   data2 rkey  group_val
0      0    a        3.5
1      1    b        7.0

* DataFrame has a more convenient join instance for merging by index. 
* It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns.

In [227]:
df6 = df4.copy()
df6.index = df6['rkey']
df6.join(df5, how='outer')

   data2 rkey  group_val
a      0    a        3.5
b      1    b        7.0
d      2    d        NaN

## Concatenation

* A common data manipulation is appending rows or columns to a dataset that already conform to the dimensions of the exsiting rows or colums, respectively. In NumPy, this is done either with `concatenate` `vstack`, `np.hstack`.

In [228]:
np.concatenate([np.random.random(5), np.random.random(5)])

array([0.12158175, 0.09856507, 0.48095483, 0.23045218, 0.45922531,
       0.99578848, 0.1138037 , 0.99722971, 0.11729559, 0.74239983])

In [229]:
np.hstack([np.random.random(5), np.random.random(5)])

array([0.3913351 , 0.54617912, 0.29700321, 0.68903758, 0.76088209,
       0.81998056, 0.58324589, 0.7286154 , 0.37406024, 0.86195248])

* This operation is also called *binding* or *stacking*.<br>
<br>
* With Pandas' indexed data structures, there are additional considerations as the overlap in index values between two data structures affects how they are concatenate.<br>

In [230]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

* If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):

In [231]:
pd.concat([s1, s2, s3], axis=1)

     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0

* In this case there is no overlap on the other axis, which as you can see is the sorted union (the 'outer' join) of the indexes. You can instead intersect them by passing join='inner':

In [232]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [233]:
pd.concat([s1, s4], axis=1, join='inner')

   0  1
a  0  0
b  1  5

* One issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the keys argument:

In [234]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result.unstack()

         a    b    c    d    e    f    g
one    0.0  1.0  NaN  NaN  NaN  NaN  NaN
two    NaN  NaN  2.0  3.0  4.0  NaN  NaN
three  NaN  NaN  NaN  NaN  NaN  5.0  6.0

## Reshaping and Pivoting

* There are a number of fundamental operations for rearranging tabular data. These are alternatingly referred to as reshape or pivot operations.

### Reshaping with Hierarchical Indexing

* Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:<br>
    * stack: this “rotates” or pivots from the columns in the data to the rows<br>
    <br>
    * unstack: this pivots from the rows into the columns

In [235]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Istanbul', 'Ankara', 'Izmir'],
                 columns=['one', 'two', 'three', 'four'])
data

          one  two  three  four
Istanbul    0    1      2     3
Ankara      4    5      6     7
Izmir       8    9     10    11

* Using the stack method on this data pivots the columns into the rows, producing a Series:

In [236]:
data.stack()

Istanbul  one       0
          two       1
          three     2
          four      3
Ankara    one       4
          two       5
          three     6
          four      7
Izmir     one       8
          two       9
          three    10
          four     11
dtype: int64

In [237]:
data.stack().unstack()

          one  two  three  four
Istanbul    0    1      2     3
Ankara      4    5      6     7
Izmir       8    9     10    11

* By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:

In [238]:
data

          one  two  three  four
Istanbul    0    1      2     3
Ankara      4    5      6     7
Izmir       8    9     10    11

In [239]:
data.stack().unstack(0)

       Istanbul  Ankara  Izmir
one           0       4      8
two           1       5      9
three         2       6     10
four          3       7     11

### Pivoting “long” to “wide” Format

* A common way to store multiple time series in databases and CSV is in so-called long or stacked format:

In [240]:
data = pd.DataFrame({'date': ["1959-03-31 00:00:00", "1959-03-31 00:00:00", "1959-03-31 00:00:00", 
                              "1959-06-30 00:00:00", "1959-06-30 00:00:00", "1959-06-30 00:00:00", 
                              "1959-09-30 00:00:00", "1959-09-30 00:00:00", "1959-09-30 00:00:00", 
                              "1959-12-31 00:00:00"],
                    'item': ['realgdp','infl','unemp','realgdp','infl','unemp','realgdp','infl','unemp','realgdp'],
                    'value': [2710.349, 0.000, 5.800, 2778.801, 2.340, 5.100, 2775.488, 2.740, 5.300, 2785.204]
                    })
data

                  date     item     value
0  1959-03-31 00:00:00  realgdp  2710.349
1  1959-03-31 00:00:00     infl     0.000
2  1959-03-31 00:00:00    unemp     5.800
3  1959-06-30 00:00:00  realgdp  2778.801
4  1959-06-30 00:00:00     infl     2.340
5  1959-06-30 00:00:00    unemp     5.100
6  1959-09-30 00:00:00  realgdp  2775.488
7  1959-09-30 00:00:00     infl     2.740
8  1959-09-30 00:00:00    unemp     5.300
9  1959-12-31 00:00:00  realgdp  2785.204

* Data is frequently stored this way in relational databases like MySQL as a fixed schema(column names and data types) allows the number of distinct values in the item column to increase or decrease as data is added or deleted in the table.<br>
<br>
* In the above example date and item would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins and programmatic queries in many cases.<br>
<br>
* The downside, of course, is that the data may not be easy to work with in long format; you might prefer to have a DataFrame containing one column per distinct item value indexed by timestamps in the date column. DataFrame’s pivot method performs exactly this transformation:

In [241]:
pivoted = data.pivot('date', 'item', 'value')
pivoted

item                 infl   realgdp  unemp
date                                      
1959-03-31 00:00:00  0.00  2710.349    5.8
1959-06-30 00:00:00  2.34  2778.801    5.1
1959-09-30 00:00:00  2.74  2775.488    5.3
1959-12-31 00:00:00   NaN  2785.204    NaN

* Note that pivot is just a shortcut for creating a reshaping with unstack.

## Data Transformation

* So far in this chapter we’ve been concerned with rearranging data.<br>
<br>
* Filtering, cleaning, and other tranformations are another class of important operations.

### Removing Duplicates

* Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [242]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,'k2': [1, 1, 2, 3, 3, 4, 4]})
data

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4

* The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate or not:

In [243]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

* Also, we can use this as a mask for finding duplicates.

In [244]:
data[data.duplicated()]

    k1  k2
1  one   1
4  two   3
6  two   4

* Other way to get duplicated values is using drop_duplicates which returns a DataFrame where the duplicated array is True

In [245]:
data.drop_duplicates()

    k1  k2
0  one   1
2  one   2
3  two   3
5  two   4

* Also, you can filter duplicates based on a column. 

In [246]:
data.drop_duplicates(['k1'])

    k1  k2
0  one   1
3  two   3

## Transforming Data Using a Function or Mapping

### Mapping

* For many data sets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

* Consider the following hypothetical data collected about customers.

In [247]:
data = pd.DataFrame({'names': ['ali', 'Burcu', 'Volkan', 'Filiz', 'Caner', 
                               'Emre', 'Dilek', 'Gamze'],
                    'birthYear': [1984, 1993, 1992, 1976, 1985, 1978, 1973, 1995]})
data

   birthYear   names
0       1984     ali
1       1993   Burcu
2       1992  Volkan
3       1976   Filiz
4       1985   Caner
5       1978    Emre
6       1973   Dilek
7       1995   Gamze

* And consider that the following hypothetical data also collected about same customers from different source.

In [248]:
birthPlaces = {'ali': 'Istanbul', 'burcu': 'Bursa', 'volkan': 'Bursa', 
                  'filiz': 'Ankara', 'caner': 'Izmir', 'emre': 'Izmir',
                  'dilek': 'Antalya', 'gamze': 'Istanbul'
}
birthPlaces

{'ali': 'Istanbul',
 'burcu': 'Bursa',
 'caner': 'Izmir',
 'dilek': 'Antalya',
 'emre': 'Izmir',
 'filiz': 'Ankara',
 'gamze': 'Istanbul',
 'volkan': 'Bursa'}

Suppose you wanted to add a column to `DataFrame` which indicate the birth place of customers.

In [249]:
data['birthPlace'] = data['names'].map(str.lower).map(birthPlaces)
data

   birthYear   names birthPlace
0       1984     ali   Istanbul
1       1993   Burcu      Bursa
2       1992  Volkan      Bursa
3       1976   Filiz     Ankara
4       1985   Caner      Izmir
5       1978    Emre      Izmir
6       1973   Dilek    Antalya
7       1995   Gamze   Istanbul

* We can also do same operation with lambda expressions.

In [250]:
data['names'].map(lambda x: birthPlaces[x.lower()])

0    Istanbul
1       Bursa
2       Bursa
3      Ankara
4       Izmir
5       Izmir
6     Antalya
7    Istanbul
Name: names, dtype: object

### Replacing Values

* Filling in missing data with the fillna method can be thought of as a special case of more general value replacement.<br>
<br>
* While map, as you’ve seen above, can be used to modify a subset of values in an object.<br>
<br>
* Replace provides a simpler and more flexible way to do so.

In [251]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

* The -999 values might be sentinel values for missing data. 
* To replace these with NaN values that pandas understands, we can use replace.

In [252]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

* Notice that replace method return new Series.

In [253]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [254]:
data = data.replace(-999, np.nan)
data

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

### Renaming Axis Indexes

* Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.<br>
<br>
* The axes can also be modified in place without creating a new data structure.

In [255]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Istanbul', 'Ankara', 'Izmir'],
                 columns=['one', 'two', 'three', 'four'])
data

          one  two  three  four
Istanbul    0    1      2     3
Ankara      4    5      6     7
Izmir       8    9     10    11

* Like a Series, the axis indexes have a map method.

In [256]:
data.index.map(str.upper)

Index([u'ISTANBUL', u'ANKARA', u'IZMIR'], dtype='object')

In [257]:
data.index = data.index.map(str.upper)
data

          one  two  three  four
ISTANBUL    0    1      2     3
ANKARA      4    5      6     7
IZMIR       8    9     10    11

### Discretization

* Pandas' `cut` function can be used to group continuous or countable data in to bins.<br>
<br>
* Discretization is generally a very **bad idea** for statistical analysis, so use this function responsibly!

* Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [258]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

* Let’s divide these into bins of 18 to 25, 26 to 35, 35 to 60, and finally 60 and older. To do so, you have to use cut, a function in pandas:

In [259]:
bins = [18, 25, 35, 60,100]

In [260]:
groups = pd.cut(ages, bins)
groups

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

* The object pandas returns is a special Categorical object.<br>
<br>
* You can treat it like an array of strings indicating the bin name; internally it contains a levels array indicating the distinct category names along with a labeling for the ages data in the codes attribute:

In [261]:
groups.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [262]:
groups.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [263]:
pd.value_counts(groups)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

* You can also pass your own bin names by passing a list or array to the labels option:

In [264]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [265]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

* A closely related function, qcut, bins the data based on sample quantiles.

In [266]:
data = np.random.randn(1000)

In [267]:
quantiles = pd.qcut(data, 4)

In [268]:
pd.value_counts(quantiles)

(0.666, 3.798]      250
(0.039, 0.666]      250
(-0.607, 0.039]     250
(-3.408, -0.607]    250
dtype: int64

### Permutation and sampling

* For some data analysis tasks, such as simulation, we need to be able to randomly reorder our data, or draw random values from it. 
* Calling NumPy's `permutation` function with the length of the sequence you want to permute generates an array with a permuted sequence of integers, which can be used to re-order the sequence.

In [269]:
sampler = np.random.permutation(5)
sampler

array([4, 1, 0, 2, 3])

* That array can then be used in the take function:

In [270]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
df

    0   1   2   3
0   0   1   2   3
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15
4  16  17  18  19

In [271]:
df.take(sampler)

    0   1   2   3
4  16  17  18  19
1   4   5   6   7
0   0   1   2   3
2   8   9  10  11
3  12  13  14  15

## Data aggregation and GroupBy operations

One of the most powerful features of Pandas is its **GroupBy** functionality. On occasion we may want to perform operations on *groups* of observations within a dataset. For exmaple:<br>
* **aggregation**, such as computing the sum of mean of each group, which involves applying a function to each group and returning the aggregated results<br>
<br>
* **slicing** the DataFrame into groups and then doing something with the resulting slices (*e.g.* plotting)<br>
<br>
* group-wise **transformation**, such as standardization/normalization

* A common data analysis procedure is the **split-apply-combine** operation, which groups subsets of data together, applies a function to each of the groups, then recombines them into a new data table.<br>
<br>
* For example, we may want to aggregate our data with with some function.

![split-apply-combine](images/groupByVisual.png)
<div align="right">*(figure taken from "Python for Data Analysis", p.251)*</div>

*  To get started, here is a very simple small tabular dataset as a DataFrame:

In [272]:
df = pd.DataFrame({
    'key1' : ['a', 'a', 'b', 'b', 'a'],
    'key2' : ['one', 'two', 'one', 'two', 'one'],
    'data1' : np.random.randn(5), 
    'data2' : np.random.randn(5)
})
df

      data1     data2 key1 key2
0  0.619755 -1.852475    a  one
1 -0.990023  0.152956    a  two
2  0.527230 -1.175463    b  one
3 -1.266556 -0.236212    b  two
4  0.277437 -1.481521    a  one

* Suppose you wanted to compute the mean of the data1 column using the groups labels from key1.
* There are a number of ways to do this. One is to access data1 and call groupby with the column (a Series) at key1.

In [273]:
grouped = df['data1'].groupby(df['key1'])
grouped

* This grouped variable is now a GroupBy object.
* It has not actually computed anything yet except for some intermediate data about the group key df['key1']. 
* The idea is that this object has all of the information needed to then apply some operation to each of the groups.(lazy evulation)

* For example, to compute group means we can call the GroupBy’s mean method:

In [274]:
grouped.mean()

key1
a   -0.030944
b   -0.369663
Name: data1, dtype: float64

In [275]:
df.groupby(['key1', 'key2']).mean()

              data1     data2
key1 key2                    
a    one   0.448596 -1.666998
     two  -0.990023  0.152956
b    one   0.527230 -1.175463
     two  -1.266556 -0.236212

* The `add_prefix` and `add_suffix` methods can be used to give the columns of the resulting table labels that reflect the transformation:

In [276]:
df.groupby(['key1', 'key2']).mean().add_suffix('_mean')

           data1_mean  data2_mean
key1 key2                        
a    one     0.448596   -1.666998
     two    -0.990023    0.152956
b    one     0.527230   -1.175463
     two    -1.266556   -0.236212

In [277]:
grouped.mean().add_prefix('MeanOf')

key1
MeanOfa   -0.030944
MeanOfb   -0.369663
Name: data1, dtype: float64

* The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [278]:
for name, group in df.groupby('key1'): 
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  0.619755 -1.852475    a  one
1 -0.990023  0.152956    a  two
4  0.277437 -1.481521    a  one
b
      data1     data2 key1 key2
2  0.527230 -1.175463    b  one
3 -1.266556 -0.236212    b  two


* In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [279]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print group

('a', 'one')
      data1     data2 key1 key2
0  0.619755 -1.852475    a  one
4  0.277437 -1.481521    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.990023  0.152956    a  two
('b', 'one')
     data1     data2 key1 key2
2  0.52723 -1.175463    b  one
('b', 'two')
      data1     data2 key1 key2
3 -1.266556 -0.236212    b  two
